In [43]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

from mods import gmap

In [44]:
load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

sql = "SELECT * FROM lodging_placeid"

df = pd.read_sql(sql, engine)

In [45]:
df_temp = (df.iloc[10:25]).copy()

df_temp

,id,name,buss_status,place_id,update_date
10,lodging_raw_10,派特屋寵物生活館,OPERATIONAL,ChIJ_8P8-BQDaDQRlA9UYkardrQ,2025/10/24
11,lodging_raw_11,Gooddog 妙狗 寵物游泳池,OPERATIONAL,ChIJyzVj03sBaDQRwtdz_lfKHbE,2025/10/24
12,lodging_raw_12,艾森寵物沙龍（全預約）,OPERATIONAL,ChIJDVa4_sIeaDQR5ELpSrMK2zY,2025/10/24
13,lodging_raw_13,聽貓說 貓咪旅館,OPERATIONAL,ChIJqUGpC8oCaDQR6K_m5VEf790,2025/10/24
14,lodging_raw_14,卡澄寵物美容寵物住宿,OPERATIONAL,ChIJg5F9ItAdaDQRspklAbqwKC4,2025/10/24
15,lodging_raw_15,PETS IN the WOOF,OPERATIONAL,ChIJK0WkAnEcaDQRs86lcetmnoE,2025/10/24
16,lodging_raw_16,Miona寵物美容,OPERATIONAL,ChIJ4S0euR0daDQRJc13lckPxss,2025/10/24
17,lodging_raw_17,布魯漫點寵物行為教室,OPERATIONAL,ChIJJThBTA8DaDQRY11UNlpJBmg,2025/10/24
18,lodging_raw_18,布魯漫點寵物旅館 青和二館,OPERATIONAL,ChIJ5ZoEcAADaDQRo5KX4FsON0k,2025/10/24
19,lodging_raw_19,金吉兒寵物沙龍旅店,OPERATIONAL,ChIJs7PiG0gCaDQRNoGwtaA2dog,2025/10/24


In [46]:
# 根據place id抓取資料，並做成新的dataframe
api_key = os.getenv("GMAP_KEY2")

id_list = df_temp["place_id"].values

data = []
for id_ in id_list:
    place_info = gmap.get_place_dict(api_key=api_key, place_id=id_)
    data.append(place_info)

df_search = pd.DataFrame(data=data)

In [ ]:
# 將要合併的欄位取出，並與原本的df合併
df_search = df_search[['place_id', 'address', 'phone', 'opening_hours', 'rating',
                       'rating_total', 'longitude', 'latitude', 'map_url', 'website', 'newest_review']]

df_combine = df_temp.merge(df_search, how="left", on="place_id")

In [67]:
# 新增類別
df_combine["category"] = "寵物旅館"

In [68]:
# 先取出正常地址格式中的市和區
pattern1 = r"([^\d\s]{2}市)([^\d\s]{1,2}區)"

df_combine[["city", "district"]] = df_combine["address"].str.extract(pattern1)

In [69]:
# 再取出反向地址格式中的市和區
pattern2 = r"([^\d\s]{2}區)([^\d\s]{2}市)"

mask = df_combine["city"].isna() & df_combine["district"].isna()
extracted = df_combine.loc[mask, "address"].str.extract(pattern2)
df_combine.loc[mask, ["district", "city"]] = extracted.values

In [70]:
# 讀取地區資料，並與店家表join，留下loc id，並將city和district欄位去除
sql_loc = "SELECT * FROM location"

df_loc = pd.read_sql(sql_loc, engine)

df_loc_select = df_loc[["loc_id", "city", "district"]]

df_final = df_combine.merge(df_loc_select, how="left", on=["city", "district"])

df_final.drop(columns=["city", "district"], axis=1, inplace=True)

In [ ]:
# 讀取類別資料，並與店家表join，留下類別id，將類別欄位去除
sql_catego = "SELECT * FROM Category"

df_catego = pd.read_sql(sql_catego, engine)

df_final = df_final.merge(df_catego, how="left",
                          left_on="category", right_on="category_name")

df_final.drop(columns=["category_name", "category"], axis=1, inplace=True)

In [ ]:
new_col = ['id', 'name', 'buss_status', 'loc_id', 'address', 'phone', 'opening_hours', 'category_id', 'rating',
           'rating_total', 'newest_review', 'longitude', 'latitude', 'map_url', 'website', 'place_id', 'update_date']

df_final = df_final[new_col]

In [73]:
df_final

,id,name,buss_status,loc_id,address,phone,opening_hours,category_id,rating,rating_total,newest_review,longitude,latitude,map_url,website,place_id,update_date
0,lodging_raw_10,派特屋寵物生活館,OPERATIONAL,NTP007,231台灣新北市新店區如意街1巷8號1樓,0222151210,"[星期一: 10:00 – 22:00, 星期二: 休息, 星期三: 10:00 – 22:...",3,5.0,10.0,2021-12-06,121.496029,24.949361,https://maps.google.com/?cid=13003769292864622484,None,ChIJ_8P8-BQDaDQRlA9UYkardrQ,2025/10/24
1,lodging_raw_11,Gooddog 妙狗 寵物游泳池,OPERATIONAL,NTP007,231台灣新北市新店區永興路10-1號,0226660178,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...",3,4.4,604.0,2025-10-07,121.545430,24.940377,https://maps.google.com/?cid=12762579398317823938,http://www.facebook.com/gooddogcaf,ChIJyzVj03sBaDQRwtdz_lfKHbE,2025/10/24
2,lodging_raw_12,艾森寵物沙龍（全預約）,OPERATIONAL,TYN002,330台灣桃園市桃園區樹仁三街79號,None,"[星期一: 10:30 – 19:00, 星期二: 10:30 – 19:00, 星期三: ...",3,5.0,44.0,2021-03-06,121.322431,24.974475,https://maps.google.com/?cid=3952764863119966948,https://instagram.com/awesome3637335?igshid=Mz...,ChIJDVa4_sIeaDQR5ELpSrMK2zY,2025/10/24
3,lodging_raw_13,聽貓說 貓咪旅館,OPERATIONAL,NTP014,236台灣新北市土城區中正路67巷33號,0905618258,"[星期一: 11:30 – 20:30, 星期二: 11:30 – 20:30, 星期三: ...",3,5.0,344.0,2025-06-19,121.444094,24.976495,https://maps.google.com/?cid=15992035238425833448,https://www.facebook.com/profile.php?id=100093...,ChIJqUGpC8oCaDQR6K_m5VEf790,2025/10/24
4,lodging_raw_14,卡澄寵物美容寵物住宿,OPERATIONAL,NTP014,236台灣新北市土城區福仁街14號1樓,0282733068,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...",3,4.8,51.0,2024-09-07,121.440146,24.976044,https://maps.google.com/?cid=3326102637742758322,https://www.facebook.com/profile.php?id=100090...,ChIJg5F9ItAdaDQRspklAbqwKC4,2025/10/24
5,lodging_raw_15,PETS IN the WOOF,OPERATIONAL,NTP008,238台灣新北市樹林區佳園路三段173號,0226686999,"[星期一: 休息, 星期二: 10:00 – 18:00, 星期三: 10:00 – 18:...",3,4.1,80.0,2023-09-03,121.385577,24.948225,https://maps.google.com/?cid=9340015838623026867,None,ChIJK0WkAnEcaDQRs86lcetmnoE,2025/10/24
6,lodging_raw_16,Miona寵物美容,OPERATIONAL,NTP008,238台灣新北市樹林區太順街110巷7號,0286853500,"[星期一: 10:00 – 19:00, 星期二: 休息, 星期三: 休息, 星期四: 10...",3,4.9,48.0,2021-08-28,121.415919,24.977952,https://maps.google.com/?cid=14683440993558580517,https://www.facebook.com/Miona1314/,ChIJ4S0euR0daDQRJc13lckPxss,2025/10/24
7,lodging_raw_17,布魯漫點寵物行為教室,OPERATIONAL,NTP014,236台灣新北市土城區青和街333號1樓,0282733464,"[星期一: 11:00 – 18:30, 星期二: 11:00 – 18:30, 星期三: ...",3,5.0,127.0,2025-07-16,121.457503,24.979783,https://maps.google.com/?cid=7495759281612152163,https://www.bluemondaypet.com/,ChIJJThBTA8DaDQRY11UNlpJBmg,2025/10/24
8,lodging_raw_18,布魯漫點寵物旅館 青和二館,OPERATIONAL,NTP014,236台灣新北市土城區青和街329號,None,None,3,NaN,NaN,None,121.457640,24.979592,https://maps.google.com/?cid=5275701276260668067,None,ChIJ5ZoEcAADaDQRo5KX4FsON0k,2025/10/24
9,lodging_raw_19,金吉兒寵物沙龍旅店,OPERATIONAL,NTP007,231台灣新北市新店區安康路二段77號,0222111511,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...",3,4.2,65.0,2022-05-03,121.513722,24.962963,https://maps.google.com/?cid=9833106900280967478,https://m.facebook.com/gingerpetshop/,ChIJs7PiG0gCaDQRNoGwtaA2dog,2025/10/24
